In [10]:
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import re

In [11]:
def getSearchPhrases():
    scope = ['https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
    client = gspread.authorize(creds)
    sheet = client.open("DIGHUM Sentiment")
    sheet = sheet.worksheet("searches")
    df = pd.DataFrame(sheet.get_all_values())
    if not df.empty:
        df.columns = df.iloc[0]
        df = df.iloc[1:]
    return df
searchPhrasesDf = getSearchPhrases()
searchPhrasesDf

,Phrases
1,Black Lives Matter
2,All Lives Matter
3,Blue Lives Matter


In [12]:
articlesDf = pd.DataFrame(columns=["query", "date","source", "title", "description"])
for query in searchPhrasesDf ["Phrases"]:
    print(query)
    preppedQuery = re.sub(" ", "%20", query)
    url = "https://news.google.com/rss/search?q=" + preppedQuery + "when:1d"
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib') 
    dates = soup.find_all('pubdate')
    titles = soup.find_all('title')
    descriptions = soup.find_all('description')
    sources = soup.find_all('source')
    for x in range(1, len(dates) + 1):
        articlesDf = articlesDf.append({"query" : query,
                                        "date" : dates[x - 1].getText(),
                                        "source": sources[x - 1].next_sibling,
                                        'title': titles[x].getText(), 
                                        "description": re.split('>', descriptions[x].get_text())[1][:-3]}, ignore_index=True)
articlesDf

Black Lives Matter
https://news.google.com/rss/search?q=Black%20Lives%20Matterwhen:1d
All Lives Matter
https://news.google.com/rss/search?q=All%20Lives%20Matterwhen:1d
Blue Lives Matter
https://news.google.com/rss/search?q=Blue%20Lives%20Matterwhen:1d


,query,date,source,title,description
0,Black Lives Matter,"Sun, 05 Jul 2020 20:30:25 GMT",The Associated Press,Facebook groups pivot to attacks on Black Live...,Facebook groups pivot to attacks on Black Live...
1,Black Lives Matter,"Sun, 05 Jul 2020 13:00:13 GMT",The Verge,33 powerful Black Lives Matter murals - The Verge,33 powerful Black Lives Matter murals
2,Black Lives Matter,"Sun, 05 Jul 2020 16:32:04 GMT",ABC News,Protester dies after struck by speeding car at...,Protester dies after struck by speeding car at...
3,Black Lives Matter,"Sun, 05 Jul 2020 20:26:19 GMT",NBC Bay Area,2 People Paint Over Black Lives Matter Mural i...,2 People Paint Over Black Lives Matter Mural i...
4,Black Lives Matter,"Sun, 05 Jul 2020 10:46:00 GMT",The Conversation CA,Black Lives Matter movement finds new urgency ...,Black Lives Matter movement finds new urgency ...
...,...,...,...,...,...
246,Blue Lives Matter,"Sun, 05 Jul 2020 04:44:50 GMT",Creators Syndicate,"Joe Biden Meets the Press, by Debra J. Saunder...","Joe Biden Meets the Press, by Debra J. Saunders"
247,Blue Lives Matter,"Sun, 05 Jul 2020 01:06:13 GMT",Report Door,Shootings soar 205 percent after NYPD disbands...,Shootings soar 205 percent after NYPD disbands...
248,Blue Lives Matter,"Sun, 05 Jul 2020 01:01:20 GMT",Daily Mail,"Gabrielle Union, Lili Reinhart lead stars spea...","Gabrielle Union, Lili Reinhart lead stars spea..."
249,Blue Lives Matter,"Sat, 04 Jul 2020 21:00:00 GMT",The Straits Times,"Ageing and living well, Lifestyle News & Top S...","Ageing and living well, Lifestyle News & Top S..."


In [13]:
articlesDf

,query,date,source,title,description
0,Black Lives Matter,"Sun, 05 Jul 2020 20:30:25 GMT",The Associated Press,Facebook groups pivot to attacks on Black Live...,Facebook groups pivot to attacks on Black Live...
1,Black Lives Matter,"Sun, 05 Jul 2020 13:00:13 GMT",The Verge,33 powerful Black Lives Matter murals - The Verge,33 powerful Black Lives Matter murals
2,Black Lives Matter,"Sun, 05 Jul 2020 16:32:04 GMT",ABC News,Protester dies after struck by speeding car at...,Protester dies after struck by speeding car at...
3,Black Lives Matter,"Sun, 05 Jul 2020 20:26:19 GMT",NBC Bay Area,2 People Paint Over Black Lives Matter Mural i...,2 People Paint Over Black Lives Matter Mural i...
4,Black Lives Matter,"Sun, 05 Jul 2020 10:46:00 GMT",The Conversation CA,Black Lives Matter movement finds new urgency ...,Black Lives Matter movement finds new urgency ...
...,...,...,...,...,...
246,Blue Lives Matter,"Sun, 05 Jul 2020 04:44:50 GMT",Creators Syndicate,"Joe Biden Meets the Press, by Debra J. Saunder...","Joe Biden Meets the Press, by Debra J. Saunders"
247,Blue Lives Matter,"Sun, 05 Jul 2020 01:06:13 GMT",Report Door,Shootings soar 205 percent after NYPD disbands...,Shootings soar 205 percent after NYPD disbands...
248,Blue Lives Matter,"Sun, 05 Jul 2020 01:01:20 GMT",Daily Mail,"Gabrielle Union, Lili Reinhart lead stars spea...","Gabrielle Union, Lili Reinhart lead stars spea..."
249,Blue Lives Matter,"Sat, 04 Jul 2020 21:00:00 GMT",The Straits Times,"Ageing and living well, Lifestyle News & Top S...","Ageing and living well, Lifestyle News & Top S..."


In [14]:
scope = ['https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
client = gspread.authorize(creds)
sheet = client.open("DIGHUM Sentiment")
sheet = sheet.worksheet("GoogleNews")
df = pd.DataFrame(sheet.get_all_values())
if not df.empty:
    df.columns =  df.iloc[0]
    df = df.iloc[1:]
newDfForSheet = articlesDf.append(df)
newindex = [x for x in range(len(newDfForSheet["query"]))]
newDfForSheet["check"] = newindex
newDfForSheet = newDfForSheet.set_index("check")
spreadsheet_key = '1YgFoSOEt9edjE0CTmMayAN6J09zMHdI-WQu9BL758Ek'
wks_name = "GoogleNews"
d2g.upload(newDfForSheet, spreadsheet_key, wks_name, credentials=creds, row_names = False)

<Worksheet 'GoogleNews' id:1652961889>